In [1]:
import os

In [9]:
folder = "/Users/reeteshsudhakar/Documents/GitHub/utk-image-classifier/utkface-images"

files = os.listdir(folder)
# grabbing the info from each file name and printing it  
for file in files:
    counter = 0
    for item in file.split("_"):
        if len(item) < 5:
            print(item)
        counter += 1 

        if counter == 3:
            print("----------")
            continue
# TODO - modify this to grab the info and add it to a table 

9
1
2
----------
36
0
1
----------
86
1
0
----------
26
1
0
----------
1
1
2
----------
52
0
1
----------
25
1
0
----------
16
0
0
----------
27
0
3
----------
36
0
3
----------
58
0
2
----------
6
1
0
----------
35
1
2
----------
1
1
3
----------
26
1
1
----------
2
0
2
----------
53
1
0
----------
35
0
3
----------
23
1
0
----------
24
0
3
----------
26
1
2
----------
75
0
0
----------
36
1
4
----------
23
1
0
----------
50
0
0
----------
15
0
0
----------
25
0
1
----------
37
0
0
----------
10
0
1
----------
22
0
1
----------
1
0
2
----------
29
1
2
----------
26
0
1
----------
21
0
0
----------
46
0
3
----------
52
0
0
----------
26
1
0
----------
27
0
3
----------
17
0
0
----------
63
0
1
----------
34
1
1
----------
61
0
3
----------
18
1
0
----------
29
1
1
----------
25
1
1
----------
34
1
3
----------
20
0
1
----------
32
0
3
----------
34
1
4
----------
29
1
0
----------
46
1
0
----------
36
1
0
----------
34
1
0
----------
32
0
3
----------
30
0
0
----------
32
1
1
---------